# Libraries

In [5]:
import numpy as np
from scipy.stats import norm as norm
from scipy.stats import poisson as poisson
from scipy.stats import nbinom
import numpy.random as random
import matplotlib.pyplot as plt
import matplotlib as mpl
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.patches as patches
import matplotlib.ticker as plticker
from matplotlib import rc #for latex use on the plots
from matplotlib import ticker  # for labels formattings
from scipy.stats import poisson
import datetime
from os import getcwd,path
import pandas as pd
from numpy import pi as pi
from time import time as timer
%matplotlib inline
from joblib import Parallel, delayed
import multiprocessing

# Plot Configs

In [6]:
plt.style.use('ggplot')
perso_linewidth = 0.6
# This will change your default rcParams
def init_plotting():
    plt.rcParams['figure.figsize'] = (40,20)
    plt.rcParams['font.size'] = 30
    plt.rcParams['font.family'] = 'serif'
    plt.rcParams['axes.labelsize'] = plt.rcParams['font.size']
    plt.rcParams['axes.titlesize'] = 1.5*plt.rcParams['font.size']
    plt.rcParams['legend.fontsize'] = plt.rcParams['font.size']
    plt.rcParams['xtick.labelsize'] = plt.rcParams['font.size']
    plt.rcParams['ytick.labelsize'] = plt.rcParams['font.size']
    #plt.rcParams['savefig.dpi'] = 2*plt.rcParams['savefig.dpi']
    plt.rcParams['axes.linewidth'] = perso_linewidth
    plt.rcParams['savefig.dpi'] = '300'
    plt.rcParams['savefig.format'] = 'pdf'
    plt.rcParams['axes.facecolor'] = '#D3D3D3'
    plt.rcParams['axes.edgecolor'] = '0'
    plt.rcParams['axes.grid'] = True
    plt.rcParams['grid.color']='white'
    plt.rcParams['grid.linestyle'] = '-'
    plt.rcParams['grid.linewidth'] = '0.4'
    plt.rcParams['axes.axisbelow'] = True
    plt.rcParams['legend.edgecolor'] = 'black'
    plt.rcParams['legend.facecolor'] = 'white'
    plt.rcParams['lines.markersize']= 18 
    plt.rcParams['lines.markeredgewidth']= '0.1'
    plt.rcParams['lines.color']= 'r' 
    plt.rcParams['lines.marker']= 'o' 
    plt.rcParams['lines.linestyle']= '-' 
    plt.rcParams['xtick.color']= '0'
    plt.rcParams['ytick.color']= '0'
    #plt.rcParams['axes.color_cycle']= ['#3778bf', '#feb308', '#a8a495', '#7bb274', '#825f87']
    plt.gca().spines['right'].set_color('none')
    plt.gca().spines['right'].set_visible('False')
    plt.gca().spines['top'].set_visible('False')
    plt.gca().spines['top'].set_color('none')
    plt.gca().xaxis.set_ticks_position('bottom')
    plt.gca().yaxis.set_ticks_position('left')
    plt.rcParams['ytick.minor.size']= 1.5
    plt.rcParams['ytick.major.width']= perso_linewidth
    plt.rcParams['ytick.minor.width']= perso_linewidth
    plt.rcParams['xtick.major.width']= perso_linewidth
    plt.rcParams['xtick.minor.width']= perso_linewidth

init_plotting()

plt.close() # this line to avoid an empty plot showing up

### Centering the plots
from IPython.core.display import HTML
HTML("""
<style>
.output_png {
    display: table-cell;
    text-align: center;
    vertical-align: middle;
}
</style>
""")

# Stochastic Simulation Analysis

### Initial state and constants

#### Simple SIS Model

##### Simple SIS model

\begin{itemize}
\item $E_1:X \rightarrow X+1,Y \rightarrow Y-1$
\item $E_2:X \rightarrow X-1,Y \rightarrow Y+1$
\end{itemize}

In [403]:
initial_state=[30,70]

### Define events increments for Gillepsie's Direct and FR: +1/-1 increment added to each compartment when Ei fires
E1=[-1,1]
E2=[1,-1]
E=np.vstack((E1,E2))


### Define state-change vector for tau-Leap
v1=[-1,1]
v2=[1,-1]
v=np.vstack((v1,v2))


simulation_period=10  ### Years

dt=1.0

#### Boarding School SIR

In [7]:
##### Boarding School

##### Actual case count
csv_file_path=path.join(getcwd(), 'data','Boarding.csv')
df=pd.read_csv(csv_file_path,index_col=0,header=0)

t_reported=df.day
Y_reported=df.flu



# ### Define events increments for Gillepsie's Direct and FR: +1/-1 increment added to each compartment when Ei fires
#E1=[1,0,0]  ## New birth
E1=[-1,1,0] ## Infection
#E3=[-1,0,0] ## Death in S
E2=[0,-1,1] ## Recovery
#E5=[0,-1,0] ## Death in I
#E6=[0,0,-1] ## Death in R


E=np.vstack((E1,E2))


### Define state-change vector for tau-Leap: Signs before each rate
v1=[-1,0,0]
v2=[1,-1,0]
v3=[0,1,-1]
v4=[0,0,1]
v=np.vstack((v1,v2,v3,v4))

simulation_period=1/16  ### Years

dt=0.01

### Rates

In [8]:
def Ratefunc(state,Beta,Gamma,Delta):
    
    
    S=state[0]
    I=state[1]
    C=state[2]
    R=state[3]
    
    N=S+I+C+R
    #######
    
    Rates=np.zeros(0)
    Rates=np.append(Rates,[(Beta*I*S)/N])
    Rates=np.append(Rates,[Gamma*I])
    Rates=np.append(Rates,[Delta*C])
    
        
    return Rates

### $\tau$-Leap

In [77]:
def TauLeap(state,Beta,Gamma,v,dt): 
    ### Returns the new state (Xn)
    
    Rates=Ratefunc(state,Beta,Gamma,dt)
    
    Lambda=dt*Rates
    Lambda_nbin=dt*Rates
    
    if state[0]>0:
        Lambda_nbin[0]=Lambda_nbin[0]/state[0]
    if state[1]>0:
        Lambda_nbin[1]=Lambda_nbin[1]/state[1]
    if state[2]>0:
        Lambda_nbin[2]=Lambda_nbin[2]/state[2]
    
      
    probs=1-np.exp(-Lambda_nbin)
    dMs_nbin=np.zeros(3)
    
    
    
    temp=np.ones(len(state))*-1
    N=sum(state)
    counter=1
    while any(value<0 for value in temp):
            
            temp=np.copy(state)
            
#             dMs=np.random.poisson(Lambda)
#             dstate=v@dMs.T
                      
            

            for i in range(3):
                 dMs_nbin[i]=np.random.binomial(state[i], probs[i], size=1)
             
            dstate=v@dMs_nbin.T   
        
 
            
            
            temp[0]=temp[0]+dstate[0]
            temp[1]=temp[1]+dstate[1]
            temp[2]=temp[2]+dstate[2]
            temp[3]=N-temp[0]-temp[1]-temp[2]
            
        
            counter=counter+1
            
        
    
    state=np.asarray(temp)
     
    return state

In [9]:
def TauLeapSim(initial_state,v,dt,simulation_period):   ### dt:days, simulation_period:years
    state=np.array(initial_state)
    n_vars=len(initial_state)
    state.shape=(1,n_vars)
    
    simulation_period_days=simulation_period*365
    
    n=int(simulation_period_days/dt)
    
    
    t=np.array([0])
    
    
    for i in range(n):
        
        R=Ratefunc(state[-1])
        new_state=TauLeap(state[-1],R,v,dt)
        state=np.vstack((state,new_state))
        t=np.append(t,t[-1]+dt)
#         if any(state[-1]<=0):
#             break
        
        
    return state,t

# Weight Function

In [10]:
def measure(Counts,Y,arg1,arg2):
    
    tau=arg1
    nbr=1/tau/tau
    Counts[Counts==0]=1
    w=nbinom.pmf(k=Y, n=nbr, p=1/(1+Counts/nbr))
    #var=X
    #var[var==0]=1
    #mean=X
    #w=norm(loc=mean,scale=var).pdf(Y)
    #w=poisson.pmf(mu=Counts,k=Y)
    #w=nbinom.pmf(k=Y, n=C, p=Rho)
    
    return w

# Bootstraping

In [11]:
def BootStrap(S,I,C,R,Y_obs,arg1,arg2):
    
    
    pobs=measure(C,Y_obs,arg1,arg2)
    
    w=pobs/sum(pobs)
    
    
    prediction=np.vstack((S,I,C,R))
    idx=np.arange(prediction.shape[1])
    filtered_idx=np.random.choice(idx,n_particles,p=w)
    filtered=prediction[:,filtered_idx]
    S=filtered[0,:]
    I=filtered[1,:]
    C=filtered[2,:]
    R=filtered[3,:]
    

    return S,I,C,R,w,pobs

# Direct Method Simulations

In [86]:
N=764
S0=763
I0=1
C0=0
R0=0


n_particles=50000
particles=np.zeros((n_particles,3))
Beta=3.2
Gamma=1.5
Delta=0.5
tau=0.1



obs_time=t_reported.values
time=t_reported.values
time=np.insert(time,0,0)
dt=time[1]-time[0]
dt=1/5
t0=1/5
time=np.arange(0,14+dt,dt)


Y_obs=Y_reported.values
#Y_obs=np.insert(Y_obs,0,0)



counter=0


S=np.zeros((n_particles,len(time)+1))
I=np.zeros((n_particles,len(time)+1))
C=np.zeros((n_particles,len(time)+1))
R=np.zeros((n_particles,len(time)+1))

Likelihood_t=np.zeros(14)
logll=np.zeros(14)




S[:,0]=S0
I[:,0]=I0
C[:,0]=C0
R[:,0]=R0


w=np.zeros(n_particles)

## Serial processing

In [87]:
#fig=plt.figure(figsize=(20,10))
#ax=plt.subplot(111)
#observed=plt.plot(obs_time,Y_obs,'-ok',linewidth=3,label='Actual reported cases',zorder=10)

for step,t in enumerate(time):#time:
    
    if any(value<0 for value in S[:,step]):
        break
    time_before = timer()
    for Particle in range(n_particles):
        
        current_state=[S[Particle,step],I[Particle,step],C[Particle,step],R[Particle,step]]
        
        new_state=TauLeap(current_state,Beta,Gamma,v,dt)
        
        S[:,step+1]=new_state[0]
        I[:,step+1]=new_state[1]
        C[:,step+1]=new_state[2]
        R[:,step+1]=new_state[3]
        
                

    time_after = timer()
    func_time=time_after-time_before
    print("Tau-leaping for %f particles time: %f"%(n_particles,func_time))
    C_prediction=C[:,step+1]
    
    
    
    if t in obs_time:
        time_before = timer()
        S[:,step+1],I[:,step+1],C[:,step+1],R[:,step+1],w,pobs=BootStrap(S[:,step+1],I[:,step+1],C[:,step+1],R[:,step+1],Y_obs[int(t)-1],arg1=tau,arg2=0)
        time_after = timer()
        func_time=time_after-time_before
        print("Bootstraping time for %f particles: %f"%(n_particles,func_time))
        logll[int(t)-1]=np.log(np.mean(pobs))
        #P=measure(C[:,step+1],Y_obs[int(t)-1],tau,0)
       
        #Likelihood_t[int(t)-1]=np.mean(P)
        scale=1000/max(w*w*w)
        print("time,obs,mean process:",t,Y_obs[int(t)-1],np.mean(C[:,step+1]))
        #PredcitionParticles=ax.scatter(np.ones(n_particles)*(t),C_prediction,marker='o',c='grey',label='Prediction Particles (Particle sizes scaled to their relative weights)',s=scale*(w*w*w),zorder=9)   ### Prediction particles

        
        time=np.arange(0,14+2*dt,dt)
        
LL=sum(logll)  
print(sum(logll))

Tau-leaping for 50000.000000 particles time: 62.061847
Tau-leaping for 50000.000000 particles time: 66.084810


KeyboardInterrupt: 

## Parallel processing

In [88]:
#fig=plt.figure(figsize=(20,10))
#ax=plt.subplot(111)
#observed=plt.plot(obs_time,Y_obs,'-ok',linewidth=3,label='Actual reported cases',zorder=10)


num_cores = multiprocessing.cpu_count()

for step,t in enumerate(time):#time:
    
    if any(value<0 for value in S[:,step]):
        break
    time_before = timer()
    
    current_state=np.vstack((S[:,step],I[:,step],C[:,step],R[:,step])).T
    
    

    new_state=Parallel(n_jobs=num_cores)(delayed(TauLeap)(current_state[Particle,:],Beta,Gamma,v,dt) for Particle in range(n_particles))
    new_state=np.asarray(new_state)
    
    S[:,step+1]=new_state[:,0]
    I[:,step+1]=new_state[:,1]
    C[:,step+1]=new_state[:,2]
    R[:,step+1]=new_state[:,3]
        
        

    time_after = timer()
    func_time=time_after-time_before
    print("Tau-leaping for %f particles time: %f"%(n_particles,func_time))
    C_prediction=C[:,step+1]
    
    
    
    if t in obs_time:
        time_before = timer()
        S[:,step+1],I[:,step+1],C[:,step+1],R[:,step+1],w,pobs=BootStrap(S[:,step+1],I[:,step+1],C[:,step+1],R[:,step+1],Y_obs[int(t)-1],arg1=tau,arg2=0)
        time_after = timer()
        func_time=time_after-time_before
        print("Bootstraping time for %f particles: %f"%(n_particles,func_time))
        logll[int(t)-1]=np.log(np.mean(pobs))
        #P=measure(C[:,step+1],Y_obs[int(t)-1],tau,0)
       
        #Likelihood_t[int(t)-1]=np.mean(P)
        scale=1000/max(w*w*w)
        print("time,obs,mean process:",t,Y_obs[int(t)-1],np.mean(C[:,step+1]))
        #PredcitionParticles=ax.scatter(np.ones(n_particles)*(t),C_prediction,marker='o',c='grey',label='Prediction Particles (Particle sizes scaled to their relative weights)',s=scale*(w*w*w),zorder=9)   ### Prediction particles

        
        time=np.arange(0,14+2*dt,dt)
        
LL=sum(logll)  
print(sum(logll))

Tau-leaping for 50000.000000 particles time: 3.676531
Tau-leaping for 50000.000000 particles time: 2.664212
Tau-leaping for 50000.000000 particles time: 2.702446
Tau-leaping for 50000.000000 particles time: 2.636783
Tau-leaping for 50000.000000 particles time: 2.710520


KeyboardInterrupt: 

## Plots

In [14]:
# for Particle in range(n_particles):
#     filtered_particles=ax.plot(time,C[Particle,:],'--or',alpha=0.5,zorder=1,label='Filtered Particles',markersize=3)

# plt.xlim((0,15))
# plt.ylim((-10,500))
# plt.legend(handles=[filtered_particles[0],PredcitionParticles,observed[0]],loc=0,fontsize=20)
# plt.xlabel('Time (days)')
# plt.ylabel('Convalescents')
# plt.title(r'Particle filtering for boarding school ($\beta$=%.2f, $\gamma$=%.2f, $\delta$=%.2f) - LogLikelihood=%.2f' %(Beta,Gamma,Delta,LL),fontsize=20)
# plt.show()
#LogLikelihood=sum(np.log(Likelihood_t))
#print(LogLikelihood)
# print(sum(logll))

logll